# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [12]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木板路穿越广阔、绿意盎然的草地，仿佛引导观者深入自然之中。图中的天空明朗，云朵稀疏，呈现出宁静愉悦的氛围。木板路两侧的草地郁郁葱葱，颜色鲜明，生机勃勃，与蓝天形成鲜明对比。这种景观常见于公园或自然保护区，旨在为访客提供探索自然之美的安全通道，同时保护地面植被。整体而言，这是一幅表现自然风光和人类与自然和谐共处的美丽图像。', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})


In [13]:
response.choices[0].message.content

'这幅图展示了一条木板路穿越广阔、绿意盎然的草地，仿佛引导观者深入自然之中。图中的天空明朗，云朵稀疏，呈现出宁静愉悦的氛围。木板路两侧的草地郁郁葱葱，颜色鲜明，生机勃勃，与蓝天形成鲜明对比。这种景观常见于公园或自然保护区，旨在为访客提供探索自然之美的安全通道，同时保护地面植被。整体而言，这是一幅表现自然风光和人类与自然和谐共处的美丽图像。'

### 封装成一个函数 query_image_description

In [14]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [15]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图非常有趣，它展示了两种不同类型的化身，被调侃地形容为“16岁的我”与“工作后的我”。左边是一只肌肉发达的狗，形似一名健身运动员，上方标注为“16岁的我”，下面有标签说明它具有强壮的身体和力量，强调爱美丽与年轻的特点。右边是一只看起来有些憔悴和消瘦的狗，被标记为“工作后的我”，其相关标签则强调工作压力和生活的艰辛，展示了疲惫和困倦的状态。整体上，这幅图通过幽默和夸张来表达人在青少年时期与成年工作后可能出现的身心变化。这种插图通常用于社交媒体上以幽默方式表达生活感悟。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [1]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [2]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图显示了1980年至2020年期间美国、中国、日本和德国的国内生产总值（GDP）的比较。从图中可以看出以下几点：

1. **美国（蓝线）**：美国GDP持续增长，1980年约为2.5万亿美元，并在40年的时间里稳步上升至超过20万亿美元。

2. **中国（红线）**：中国的GDP增长非常显著。从1980年的几乎接近于0万亿美元，到2020年增长至约14万亿美元。特别是从2000年开始，中国GDP的增长速度加快，显示出迅猛的经济发展。

3. **日本（紫线）**：日本的GDP在1980年到1995年期间稳步增长，达到约5万亿美元的高峰。然而，自那时以后，日本的GDP增长放缓，并在5万亿到6万亿美元之间波动。

4. **德国（绿线）**：德国的GDP增长较为平稳，从1980年的不到1万亿美元增长至约4万亿美元。

整体来看，这幅图表反映了这四个国家在过去40年内的经济增长情况，显示美国和中国的GDP增长尤为显著，而日本和德国的增长则较为缓慢。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [3]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片展示的是一本笔记本上关于机器学习模型微调技术的笔记，特别是关于变换器模型（如BERT或GPT等）的变体方法。文中讨论了几种不同的微调技术，包括：

1. **Prompt Tuning**: This involves adjusting or optimizing a prompt to steer the outputs of a pre-trained model for specific tasks without altering the model parameters substantially. Often used with smaller models.

2. **Prefix Tuning**: In this method, a sequence of continuous vectors (prefixes) is prepended to the input, and these vectors are optimized during training while keeping the underlying model's weights frozen.

3. **LoRA (Low-Rank Adaptation)**: LoRA modifies the attention mechanism of transformer models by adding low-rank matrices to the weights. This means changes to the model are restricted to a lower-dimensional subspace, which reduces the amount of parameters that need updating.

笔记中详细记录了这些技术的数学表述和一些实现细节。例如，LoRA技术中的公式 \( Y = (W + \Delta W) X \)，其中 \( \Delta W = AB \)，说明了是如何通过低秩矩阵 \( AB \) 来近似权重的变化。还提到了一些具体的数据存储需求，例如 QLoRA 模型需要 78GB存储空间，而 LLaMA模型使用的QLoRA只需 48GB。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [4]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片展示的是一本笔记本上关于机器学习模型微调技术的笔记，特别是关于变换器模型（如BERT或GPT等）的变体方法。文中讨论了几种不同的微调技术，包括：

1. **Prompt Tuning**: This involves adjusting or optimizing a prompt to steer the outputs of a pre-trained model for specific tasks without altering the model parameters substantially. Often used with smaller models.

2. **Prefix Tuning**: In this method, a sequence of continuous vectors (prefixes) is prepended to the input, and these vectors are optimized during training while keeping the underlying model's weights frozen.

3. **LoRA (Low-Rank Adaptation)**: LoRA modifies the attention mechanism of transformer models by adding low-rank matrices to the weights. This means changes to the model are restricted to a lower-dimensional subspace, which reduces the amount of parameters that need updating.

笔记中详细记录了这些技术的数学表述和一些实现细节。例如，LoRA技术中的公式 \( Y = (W + \Delta W) X \)，其中 \( \Delta W = AB \)，说明了是如何通过低秩矩阵 \( AB \) 来近似权重的变化。还提到了一些具体的数据存储需求，例如 QLoRA 模型需要 78GB存储空间，而 LLaMA模型使用的QLoRA只需 48GB。

![](./images/handwriting_1.jpg)

In [5]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片显示的是一本笔记本的两页，里面记录了有关自然语言处理（NLP）和大型语言模型（LLMs）的一些高级主题和研究方法。内容涵盖了多种不同的调整和优化语言模型的技术：

1. 左侧页面：
   - 开头提到了`一阶Transformers技术`和`技术`下有`Benchmark`进行性能评估。
   - 下方列举了不同的模型调优方法，如`PEFT`、`SOTA`（State Of The Art，技术前沿）和`PBFT Methods`。
   - 进行了`Prompt Tuning`的特别说明，涉及不同年份和组织，例如Google和Stanford。
   - 底部提到了高级语言模型，如`GPT、XLMH、ChatGLM、BLOOM`和`Alpaca`等。

2. 右侧页面：
   - 右边页起始处提到了`multi-modality`和`Instruction Fine-Tuning`（指令微调），给出了具体的例子比如`LLaMA(33B)`。
   - 提到了新的模型`LaRa`和`PETC`，及其相关技术。
   - 研究了`Prefix-tuning`和`Adaptors`的技术性区分，并讨论了在大型语言模型中的Refine技巧。
   - 还有对`MAM Adaptors`的讨论，包括架构设计和效果提升的描述。

这些笔记显示，作者正在研究或学习关于如何通过各种方法优化或调整语言模型的技术细节。这可能是在计算机科学、特别是自然语言处理领域的高级学习或研究的一部分。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

![](./images/vita-drink.jpg)

In [8]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description2(image_path, prompt, max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

In [13]:
content = query_base64_image_description2("./images/vita-drink.jpg","此产品的类型是什么？")
display(Markdown(content))

这张图片显示的产品是豆奶粉。我们可以通过包装上的文字了解到，这是植物蛋白饮品，由大豆蛋白分离物等成分制成。此外，包装上还列出了营养成分表，包括蛋白质、碳水化合物、钠等成分的含量。